# CeleryTaskScheduler 示例（在 Jupyter 中运行）

本 Notebook 演示如何在 Jupyter 中直接导入并使用项目封装的 `CeleryTaskScheduler`。

注意：跨进程（worker + broker）演示是可选的。为了方便在单机或没有 broker 的环境下快速试验，下面提供了“内存 / eager”演示配置，使用该配置无需外部 broker 或单独 worker。

## 运行指南（可选：跨进程 或 内存/eager 演示）

1) 跨进程（broker + worker，生产或分布式测试）：

- 使用 RabbitMQ：可本地安装或用 Docker 启动 rabbitmq:3-management。
- 启动 worker（在包含任务定义模块的目录）：

```bash
# 激活 venv 后
celery -A examples.celery_tasks worker --loglevel=info
```

2) 内存 / eager 演示（单进程，适合 notebook 和快速开发）：

- 在 Notebook 中将 Celery 配置为 eager 模式即可在当前进程同步执行任务，无需外部 broker 或 worker。
- 示例：
```python
# 在 notebook 中配置 examples.celery_tasks.app 为 eager（演示/测试用途）
examples_tasks.app.conf.update(
    task_always_eager=True,
    task_eager_propagates=True,  # 让异常在调用端抛出，便于调试
)
```

- 内存演示注意事项：eager 模式会在当前进程中直接运行任务，因此适合演示与单进程测试，但不能模拟真实的分布式环境（例如任务路由、并发 worker 行为等）。

In [51]:
# 安装依赖（若未安装）
# 在 notebook 内可通过 !pip 安装，但建议在 venv 中提前安装
!uv pip install -q celery anyio
print('installed (or already present)')

installed (or already present)


In [52]:
# 在笔记本中导入并验证 astrbot_canary_api（使用封装的调度器）
try:
    from astrbot_canary_api.scheduler import CeleryTaskScheduler
    print('Imported CeleryTaskScheduler')
except Exception as e:
    print('Import failed:', e)
    raise

# 尝试导入 examples.celery_tasks（任务定义）
import importlib, pathlib
try:
    examples_tasks = importlib.import_module('examples.celery_tasks')
    print('Loaded examples.celery_tasks as module')
except Exception:
    # fallback: load from file
    spec_path = pathlib.Path.cwd() / 'celery_tasks.py'
    import importlib.util
    spec = importlib.util.spec_from_file_location('examples.celery_tasks', str(spec_path))
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)  # type: ignore[attr-defined]
    examples_tasks = mod
    print('Loaded examples.celery_tasks from file')

# verify the module provides an app and a task named 'hello'
print('has app:', hasattr(examples_tasks, 'app'))
print('has hello:', hasattr(examples_tasks, 'hello'))


Imported CeleryTaskScheduler
Loaded examples.celery_tasks from file
has app: True
has hello: True


In [53]:
# 为演示：把 Celery 配为 eager（内存/同步）模式，这样 notebook 不需要外部 broker
# 注意：eager 模式会在当前进程中直接执行任务，适合演示和单进程测试
examples_tasks.app.conf.update(
    task_always_eager=True,
    task_eager_propagates=True,  # 让异常在调用端抛出，便于调试
)
print('Celery configured for eager (in-process) execution')

# 同步示例：优先使用任务对象的 apply_async 或直接调用（在 eager 下更可靠）
from astrbot_canary_api.scheduler import CeleryTaskScheduler
sched = CeleryTaskScheduler(app=examples_tasks.app)

# 1) 使用任务对象的 apply_async（在 eager 下会同步执行）
print('--- apply_async (task object) ---')
try:
    handle_apply = sched.apply_async(examples_tasks.hello, args=('notebook-user',))
    print('apply_async id:', handle_apply.id())
    print('apply_async ready?:', handle_apply.ready())
    print('apply_async result:', handle_apply.get(timeout=5))
except Exception as e:
    print('apply_async failed:', type(e), e)


Celery configured for eager (in-process) execution
--- apply_async (task object) ---
apply_async id: 6fe936ef-fc60-4bfb-aa6b-809fb12af743
apply_async ready?: True
apply_async result: hello notebook-user


In [ ]:
# 异步示例：在 Jupyter 中使用顶层 await (兼容 eager / 非-eager)
from astrbot_canary_api.scheduler import CeleryTaskScheduler

# 在 notebook 顶层创建 scheduler，避免在每次调用中重复创建
sched = CeleryTaskScheduler(app=examples_tasks.app)

async def async_example():
    try:
        # 打印当前调度器与 app 的关键配置，帮助理解走本地还是远端路径
        print('scheduler.prefer_local =', getattr(sched, 'prefer_local', None))
        print('app.task_always_eager =', examples_tasks.app.conf.get('task_always_eager'))

        # 方法 A: 通过任务名提交（registered_only=False 允许按名称查找注册任务）
        handle = await sched.async_apply_async('examples.hello', args=('async-notebook',), registered_only=False)
        # 方法 B (等价)：
        # handle = await sched.async_send_task('examples.hello', args=('async-notebook',))

        print('async submitted id:', handle.id())
        print('handle.metadata =', getattr(handle, 'metadata', None))

        # 如果句柄已经就绪（InMemoryResultHandle 情况），优先直接取结果以避免等待后端
        ready = False
        try:
            ready = bool(handle.ready())
        except Exception:
            ready = False

        if ready:
            result = handle.get()
            print('async got result (ready):', result)
            return

        # 否则走异步等待路径（会在后台线程中调用 get，避免阻塞事件循环）
        try:
            result = await sched.async_get_result(handle.id(), timeout=10)
            print('async got result:', result)
        except Exception as e:
            print('async_example failed during backend wait:', type(e), e)

    except Exception as e:
        # 在 eager 模式下若 task_eager_propagates=True，异常会直接在这里抛出并被捕获
        print('async_example failed:', type(e), e)

# Jupyter 支持顶层 await；直接 await 可避免 'Already running asyncio' 问题
await async_example()


async submitted id: 4d854878-b484-4882-85c5-337b12174658
async_example failed: <class 'celery.exceptions.TimeoutError'> The operation timed out.
async_example failed: <class 'celery.exceptions.TimeoutError'> The operation timed out.


## 故障排查 & 小贴士

- 如果看到 `Unknown task` 错误：确认 worker 已加载包含任务定义的模块（启动 worker 时的工作目录与 PYTHONPATH）。
- 若无法获取结果：检查 Celery 的 `backend` 配置（需要设置 result backend 才能使用 AsyncResult.get）。
- 在 VS Code 中，确保 Notebook kernel 使用的是与启动 worker 相同的虚拟环境（否则导入路径或依赖可能不一致）。

常用命令复习：

```bash
# 启动 worker（在项目根，确保 examples/celery_tasks.py 在 path 中）
celery -A examples.celery_tasks worker --loglevel=info

# 如果使用 redis:
celery -A examples.celery_tasks worker --broker=redis://localhost:6379/0 --loglevel=info
```
